In [1]:
# TP 4 - Options barrières

import numpy as np
from scipy.stats import binom

n = 5 # nombre d'étapes
T = 5# temps final
deltat = T/n # pas de temps
S0 = 100 # prix initial
# sigma = 0.1 # volatilité
up = 1.25 # up
down = 1/up # down

# taux d'intérêt et facteur d'actualisation
r = 0
R = np.exp(r*deltat)

# probabilité risque neutre
p = (R-down)/(up-down)

print("p =",p)

p = 0.44444444444444436


In [2]:
# matrice des prix de l'actif (TP1)
def CRR(n,down,up,S0):
    S = np.zeros((n+1,n+1))
    S[0,0] = S0
    for i in range(n):
        S[i+1,0] = S[i,0]*down
        for j in range(i+1):
            S[i+1,j+1] = S[i,j]*up
    return S


In [3]:
S = CRR(n,down,up,S0)
S

array([[100.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ 80.        , 125.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ 64.        , 100.        , 156.25      ,   0.        ,
          0.        ,   0.        ],
       [ 51.2       ,  80.        , 125.        , 195.3125    ,
          0.        ,   0.        ],
       [ 40.96      ,  64.        , 100.        , 156.25      ,
        244.140625  ,   0.        ],
       [ 32.768     ,  51.2       ,  80.        , 125.        ,
        195.3125    , 305.17578125]])

In [4]:
# paramètres de l'option
K = 100 # strike
L = 130 # barrière

def payoffcall(S,K):
    phicall = np.maximum(S-K,0) # option d'achat
    return phicall

def payoffput(S,K):
    phiput = np.maximum(K-S,0) # option de vente
    return phiput


In [5]:
# évaluation du prix d'un call européen par récurrence rétrograde
C = np.zeros((n+1,n+1))
for j in range(n+1):
    C[n,j] = payoffcall(S[n,j],K)
for i in range(n-1,-1,-1):
    for j in range(i+1):
        C[i,j] = (p*C[i+1,j+1]+(1-p)*C[i+1,j])/R


In [6]:
print("La prime du contrat call européen vaut C0 =",C[0,0])

La prime du contrat call européen vaut C0 = 20.66250063506578


In [7]:
# calcul des valeurs de l'option par formule binomiale directe
CC = np.zeros((n+1,n+1))
for j in range(n+1):
    CC[n,j] = payoffcall(S[n,j],K)
for i in range(n+1):
    for j in range(i+1):
        CC[i,j] = np.dot(payoffcall(S[n,j:j+n-i+1],K),binom.pmf(range(n-i+1),n-i,p))/R**(n-i)


In [8]:
print("La prime du contrat call européen vaut C0 =",CC[0,0])

La prime du contrat call européen vaut C0 = 20.662500635065776


In [17]:
# indicatrice de la région sous la barrière (option D-I-C)
# indice = 1 => on est sous L (option active); indice = 0 => on est au dessus de L
def sousL(i,j):
    if (S[i,j]<=L):
        indice = 0
    else:
        indice = 1
    return indice

In [10]:
sousL(100,47)
S[100,46:48]

IndexError: index 100 is out of bounds for axis 0 with size 6

In [18]:
# évaluation du prix d'une option barrière DIC par récurrence rétrograde
DIC = np.zeros((n+1,n+1,2)) # la 3ème entrée permet de gérer si on est déjà passé sous L
# initialisation par la valeur finale
for j in range(n+1):
    # k=0: on n'est pas passé en dessous de L (=> option inactive)
    DIC[n,j,0] = 0
    # k=1: on est passé sous L (=> option active)
    DIC[n,j,1] = payoffcall(S[n,j],K)
# récurrence rétrograde
for i in range(n-1,-1,-1):
    for j in range(i+1):
        # k = 0: on n'est pas passé sous L JUSQUE LA!
        DIC[i,j,0] = (p*DIC[i+1,j+1,0]+(1-p)*DIC[i+1,j,sousL(i+1,j)])/R
        # k = 1: on est passé sous L avant => option active
        DIC[i,j,1] = (p*DIC[i+1,j+1,1]+(1-p)*DIC[i+1,j,1])/R

In [19]:
print("La prime du contrat DIC vaut DIC0 =",DIC[0,0,0]) # on suppose S0 > L

La prime du contrat DIC vaut DIC0 = 4.809564937594199


In [ ]:
# en fait DIC[:,:,1] == C (call classique)
np.amax(abs(DIC[:,:,1]-C[:,:]))

In [20]:
# évaluation du prix d'une option barrière DOC par récurrence rétrograde
DOC = np.zeros((n+1,n+1,2)) # la 3ème entrée permet de gérer si on est déjà passé sous L
# initialisation par la valeur finale
for j in range(n+1):
    # k=0: on n'est pas passé en dessous de L (=> option toujours active)
    DOC[n,j,0] = payoffcall(S[n,j],K)
    # k=1: on est passé sous L (=> option désactivée)
    DOC[n,j,1] = 0
# récurrence rétrograde
for i in range(n-1,-1,-1):
    for j in range(i+1):
        # k = 0: on n'est pas passé sous L JUSQUE LA! (toujours active)
        DOC[i,j,0] = (p*DOC[i+1,j+1,0]+(1-p)*DOC[i+1,j,sousL(i+1,j)])/R
        # k = 1: on est passé sous L avant => option désactivée
        DOC[i,j,1] = (p*DOC[i+1,j+1,1]+(1-p)*DOC[i+1,j,1])/R

In [21]:
print("La prime du contrat DOC vaut DOC0 =",DOC[0,0,0]) # on suppose S0 > L

La prime du contrat DOC vaut DOC0 = 15.852935697471581


In [ ]:
# en fait, DOC[:,:,1] == 0
DOC[:,:,1]

In [ ]:
# on compare DIC, DOC et C
print("La prime du contrat Call vaut C0 =",C[0,0])
print("La prime du contrat DIC vaut DIC0 =",DIC[0,0,0])
print("La prime du contrat DOC vaut DOC0 =",DOC[0,0,0])
print("La somme DIC0 + DOC0 vaut",DIC[0,0,0]+DOC[0,0,0])


In [ ]:
# Calcul DIC0 par somme directe binomiale
# on veut adapter:
# CC[0,0] = np.dot(payoffcall(S[n,0:n+1],K),binom.pmf(range(n+1),n,p))/R**(n)
# indices correspondant à la barrière et au strike
indl = int(np.floor((n+np.log(L/S0)/np.log(up))/2))
indk = 1+int(np.floor((n+np.log(K/S0)/np.log(up))/2))
indJ = int(np.floor(n+np.log(L/S0)/np.log(up)))
if (K<L):
    # 1ère somme binomiale: de indk à indl (au lieu de 0 à n)
    DIC0 = np.dot(payoffcall(S[n,indk:indl+1],K),binom.pmf(range(indk,indl+1),n,p)) # première somme partielle binomiale
    # 2ème somme de indl+1 à indJ
    DIC0 += np.dot(payoffcall(S[n,indl+1:indJ+1],K),binom.pmf(range(indJ-indl-1,-1,-1),n,1-p))*(p/(1-p))**(indJ-n)# deuxième somme partielle binomiale
    # attention, il faut des coeffs binomiaux differents des puissances de p et 1-p...
    DIC0 = DIC0/R**n # actualisation


In [ ]:
print("La prime du contrat DIC vaut DIC0 =",DIC0)

In [ ]:
indJ = int(n+np.floor(np.log(L/S0)/np.log(up)))
indl = int(np.floor((n+np.log(L/S0)/np.log(up))/2))
indk = 1+int(np.floor((n+np.log(K/S0)/np.log(up))/2))
print(indJ,indl,indk)

In [ ]:
S[100,46:48]